#**GridSearchCV and RandomSearchCV**

Taking Boston house price dataset to check accuracy of Random Forest Regression model and tuning hyperparameters-number of estimators and max depth of the tree to find the best value. 

First load boston data and split into train and test sets.

In [ ]:
pip install Pipeline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np 
import pandas as pd


In [ ]:
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]

In [ ]:
#splitting data into train and test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data,target,test_size=0.3)

Apply Random Forest Regressor model with n_estimators of 5 and max_depth of 3

In [ ]:
from sklearn import ensemble
dt=ensemble.RandomForestRegressor(n_estimators=5,max_depth=3)
dt.fit(x_train,y_train)
print('training score: ', dt.score(x_train,y_train))
print('test score: ',dt.score(x_test,y_test))

training score:  0.8621744277402092
test score:  0.8077758263589873


Let's try with cross validation and see:

In [ ]:
from sklearn.model_selection import cross_val_score
scores1 = cross_val_score(ensemble.RandomForestRegressor(n_estimators=5,max_depth=3),x_train,y_train,cv=10)
np.average(scores1)

0.7334703222747299

The *Cross Validation* alone is not giving us something good. Let's combine it with *GridSearchCV*

For this, a list of dictionaries of the list of hyperparameter values is passed. Then model is evaluated on every combination of each list of values to find the best one.



In [ ]:
ensemble.RandomForestRegressor().get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [ ]:
model=ensemble.RandomForestRegressor()
from sklearn.model_selection import GridSearchCV
parameters=[{'n_estimators':[20,30,40,60,100], 'max_depth':  
             [5,10,15,20]},
             {'n_estimators':[20,30,40,60,100], 'max_depth': 
             [5,10,15,20],'max_features':[2,5,8]}]
             
grid_search = GridSearchCV(estimator=model,
                           param_grid=parameters,
                           cv=10,
                           n_jobs=-1)
                           
grid = grid_search.fit(x_train,y_train)
grid.best_score_

0.8264319910026565

In [ ]:
grid.best_params_

{'max_depth': 10, 'max_features': 5, 'n_estimators': 100}

In [ ]:
#trying the best hyperparameters suggested by GridSearchCV
#{'max_depth': 10, 'max_features': 5, 'n_estimators': 100}

from sklearn import ensemble
dt_1=ensemble.RandomForestRegressor(n_estimators=100, max_depth=10, max_features = 5)
dt_1.fit(x_train,y_train)
print('training score: ', dt_1.score(x_train,y_train))
print('test score: ',dt_1.score(x_test,y_test))

training score:  0.9765502544034178
test score:  0.8901441683241353


#Random Search CV

In Random search CV, the best combination can not be identified because all the combinations are not tested. But the advantage is we can test a broad range of values for hyperparameters within the same computation time as grid search CV.

Below is the implementation of Random search for the above example of Boston Housing prices dataset.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
model=ensemble.RandomForestRegressor()
param_grid=[{'n_estimators':[20,30,40,60,100], 'max_depth':[5,10,15,20] 
          },{'n_estimators':[20,30,40,60,100], 'max_depth':[5,10,15,20]
             ,'max_features':[2,5,8]}]

rnd_search = RandomizedSearchCV(model, param_grid, cv=10, 
                                          return_train_score=True)
rnd_search.fit(x_train,y_train)
rnd_search.best_score_

0.8235312365882501

In [ ]:
rnd_search.best_params_

{'max_depth': 10, 'max_features': 8, 'n_estimators': 20}

In [ ]:
#trying the best hyperparameters suggested by RandomizedSearchCV

#{'max_depth': 10, 'max_features': 8, 'n_estimators': 20}



from sklearn import ensemble
dt=ensemble.RandomForestRegressor(n_estimators=20, max_depth=10, max_features = 8)
dt.fit(x_train,y_train)
print('training score: ', dt.score(x_train,y_train))
print('test score: ',dt.score(x_test,y_test))

training score:  0.9644230305537048
test score:  0.881733229588773


#**Save and Load Machine Learning Models**

In Machine Learning, when we finally arrive at the model that we are satisfied with, we can save and reuse it later.

In this session, we will see the two(2) most common ways of saving our model and reloading it for future predictions:

1. Finalize Your Model with pickle
2. Finalize Your Model with Joblib

#Finalize Your Model with pickle

Pickle is the standard way of serializing objects in Python. You can use the pickle1 operation to serialize your machine learning algorithms and save the serialized format to a file. Later you can load this file to deserialize your model and use it to make new predictions. The example below demonstrates how you can save your model to file and load it to make predictions on the unseen test set. We will save the best model from our previous tutorial(see code above).

In [ ]:
import pickle


pickle.dump(dt_1, open('model.pkl', 'wb'))



# some time later...
# load the model from disk
pickled_model = pickle.load(open('model.pkl', 'rb'))

Running the example saves the model to finalized model.pkl in your local working directory. Load the saved model and evaluating it provides an estimate of accuracy of the model on unseen data.

In [ ]:
pickled_model.predict(x_test) 

array([26.83962126,  8.61232381, 20.51751326, 28.19453292, 23.94116815,
       25.2110988 , 34.93170079, 20.39077568, 16.02745032, 16.89315219,
       21.85855031, 19.75804735, 22.33963641, 14.32117489, 15.36852813,
       21.3690862 , 15.02309333, 20.38319334, 19.3126351 , 23.22964672,
       23.56209709, 21.1691938 , 41.71069231, 24.2908043 , 20.38991156,
       23.16937334, 44.898     , 16.36586787, 20.49836588, 20.66797121,
       25.87777379, 20.77371312, 20.55072132, 21.87943995, 17.85003591,
       46.557     , 26.16849376, 19.88904843, 13.01537154, 19.76115808,
       14.46104524, 23.88767507, 10.21754719, 15.71266115, 24.62951764,
       18.65706899, 27.71905513, 27.29179945, 20.29504793, 20.62034342,
       23.52518336, 20.56068088, 24.05032143, 21.01796634, 13.11094538,
       20.85114141, 27.7025    , 34.13029419, 14.11621995, 21.98597677,
       15.77697325, 24.57999068, 29.2788869 , 31.66714921,  9.13535909,
       23.38985045, 21.22257574, 32.2963918 , 11.0103697 , 20.96

#Finalize Your Model with Joblib

In [ ]:
!pip install joblib

In [ ]:
import joblib


joblib.dump(dt_1 , 'joblib_model')

['joblib_model']

In [ ]:
jobLib = joblib.load('joblib_model')



jobLib.predict(x_test)


array([26.83962126,  8.61232381, 20.51751326, 28.19453292, 23.94116815,
       25.2110988 , 34.93170079, 20.39077568, 16.02745032, 16.89315219,
       21.85855031, 19.75804735, 22.33963641, 14.32117489, 15.36852813,
       21.3690862 , 15.02309333, 20.38319334, 19.3126351 , 23.22964672,
       23.56209709, 21.1691938 , 41.71069231, 24.2908043 , 20.38991156,
       23.16937334, 44.898     , 16.36586787, 20.49836588, 20.66797121,
       25.87777379, 20.77371312, 20.55072132, 21.87943995, 17.85003591,
       46.557     , 26.16849376, 19.88904843, 13.01537154, 19.76115808,
       14.46104524, 23.88767507, 10.21754719, 15.71266115, 24.62951764,
       18.65706899, 27.71905513, 27.29179945, 20.29504793, 20.62034342,
       23.52518336, 20.56068088, 24.05032143, 21.01796634, 13.11094538,
       20.85114141, 27.7025    , 34.13029419, 14.11621995, 21.98597677,
       15.77697325, 24.57999068, 29.2788869 , 31.66714921,  9.13535909,
       23.38985045, 21.22257574, 32.2963918 , 11.0103697 , 20.96